# Лабораторная работа #5
Вариант #1

Искендеров А.С.

In [95]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [97]:
df = pd.read_csv("winequality-white.csv", sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Посмотрим, какие есть качества у вин.

In [98]:
sorted(df["quality"].unique())

[3, 4, 5, 6, 7, 8, 9]

Отметим все вина с качеством выше 6, как высококачетсвенные.

In [99]:
df["is high quality"] = (df["quality"] >= 7).astype(int)
df = df.drop(columns="quality")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is high quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


Мастштабируем данные и разделим их на тренировочные и тестовые.

In [100]:
X = df.drop(columns="is high quality")
y = df["is high quality"]

scaler = StandardScaler()
X =  scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Натренируем многослойные персептроны с различными количествами слоёв и 
нейронов на слой.

In [101]:
results = {}

for layers in [(32,), (64,), (64, 32), (128, 64, 32)]:
    model = MLPClassifier(
        hidden_layer_sizes=layers,
        max_iter=10000,
        random_state=42
    )
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)

    results[f"MLP {layers}"] = (acc, f1)

Для сравнения натренируем модели логистической регрессии и случайного 
леса.

In [102]:
models = {
    "LR": LogisticRegression(),
    "RF": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)

    results[name] = (acc, f1)

Отсортируем результаты и сравним точности моделей.

In [103]:
sorted_results = sorted(results.items(), key=lambda x: x[1][0], reverse=True)

for model_name, (acc, f1) in sorted_results:
    print(f"{model_name}:\n{acc:.2f} acc, {f1:.2f} f1\n")

RF:
0.89 acc, 0.72 f1

MLP (64, 32):
0.86 acc, 0.67 f1

MLP (128, 64, 32):
0.86 acc, 0.67 f1

MLP (64,):
0.84 acc, 0.61 f1

MLP (32,):
0.83 acc, 0.56 f1

LR:
0.80 acc, 0.38 f1

